In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
from synth_dim_model import *

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- PARAMETERS ---
N = 3
M = 2
min_V_J_ratio = -10
max_V_J_ratio = 10
max_mu_V_ratio = 10
resolution = 500

mu_V_ratios = np.linspace(0, max_mu_V_ratio, resolution)
J_V_ratios = np.linspace(min_V_J_ratio, max_V_J_ratio, resolution)
# Create a grid for the two parameters.
mu_V_ratio_grid, J_V_ratio_grid = np.meshgrid(mu_V_ratios, J_V_ratios)

# Loop over the two cases: V negative and V positive.
for V in [-1, 1]:
    # Allocate arrays to hold the computed data.
    sigma_grid = np.zeros_like(mu_V_ratio_grid)
    energy_gap_over_V_grid = np.zeros_like(mu_V_ratio_grid)
    
    # (Assuming your function returns a tuple (states, …))
    states, _ = enumerate_states(N, M)
    
    # Loop over the parameter grid (note: the meshgrid is built so that the first index corresponds to J_V_ratios).
    for i, mu_V_ratio in enumerate(mu_V_ratios):
        for j, J_V_ratio in enumerate(J_V_ratios):
            H = construct_rescaled_hamiltonian(N, M, V, mu_V_ratio, J_V_ratio)
            eigenvalues, eigenvectors = exact_diagonalize(H)
            energy_gap = eigenvalues[1] - eigenvalues[0]
            energy_gap_over_V_grid[j, i] = energy_gap
            sigma_grid[j, i] = sigma_ij(0, 1, eigenvectors[0], states, N, M) / M

    # Determine a string to indicate the sign of V for naming the files.
    sign = "positive" if V > 0 else "negative"
    
    # --- SAVE ENERGY GAP DATA ---
    # For each grid point we store: mu/|V|, J/|V|, energy_gap.
    data_energy = np.column_stack((
        mu_V_ratio_grid.ravel(), 
        J_V_ratio_grid.ravel(), 
        energy_gap_over_V_grid.ravel()
    ))
    energy_filename = f"energy_gap_V_{sign}.csv"
    np.savetxt(
        energy_filename,
        data_energy,
        delimiter=",",
        header="mu_V_ratio,J_V_ratio,energy_gap",
        comments=""
    )
    print(f"Saved energy gap data to {energy_filename}")

    # --- SAVE SIGMA DATA ---
    data_sigma = np.column_stack((
        mu_V_ratio_grid.ravel(), 
        J_V_ratio_grid.ravel(), 
        sigma_grid.ravel()
    ))
    sigma_filename = f"sigma_V_{sign}.csv"
    np.savetxt(
        sigma_filename,
        data_sigma,
        delimiter=",",
        header="mu_V_ratio,J_V_ratio,sigma",
        comments=""
    )
    print(f"Saved sigma data to {sigma_filename}")

# (Optional) You may also display a sample plot:
# Note that in your original code the energy gap was plotted as log(1/energy_gap).
# Here we do not plot immediately but you could do so if desired.
